In [ ]:
!pip install bertopic
import os
from google.colab import drive
from bertopic import BERTopic
drive.mount('/content/drive/')

In [ ]:
cd /content/drive/MyDrive

In [ ]:
#전처리 패키지
import pandas as pd
import numpy as np
from datetime import datetime
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS
import spacy
import string

#토픽 모델링 패키지
from bertopic import BERTopic
import tensorflow_hub
from umap import UMAP
import hdbscan
import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

#그 외
import warnings


In [ ]:
#MyDrive data경로에 뉴스 파일을 업로드 해줘야함
cnbc=pd.read_csv('./data/US_topics_tagged.csv')

In [ ]:
#중복제거 후 데이터 생성
#BERTopic의 경우 텍스트 전처리를 사전에 진행하는 것보다 후에 처리하는 것이 더 좋은 결과를 뽑아낼 수 있습니다.

filter_cnbc=cnbc.drop_duplicates(['date','contents'],keep='first') # 중복 제거 

filter_cnbc['date']=filter_cnbc.date.astype(str) 
filter_cnbc['date']=filter_cnbc['date'].apply(lambda x : x[:8]) 
filter_cnbc['date']=pd.to_datetime(filter_cnbc['date']) #datetime 변환

filter_cnbc[filter_cnbc['contents']=='nan'] = np.nan
filter_cnbc.dropna(subset=['contents'],how='any',axis=0,inplace=True) #결측치 제거

#filter_cnbc.contents=filter_cnbc.contents.apply(lambda x : text_preprocessing(x))    

week_cnbc=filter_cnbc[filter_cnbc.date.between('2022-08-18','2022-08-29')].sort_values('date') #원하는 날짜 조정 (주간 단위)
week_CNBC=week_cnbc.reset_index(drop=True)



In [ ]:
#뉴스 문서 테마 라벨링을 위한 sub_news data생성
cnbc['date']=cnbc.date.astype(str) 
cnbc['date']=cnbc['date'].apply(lambda x : x[:8]) 
cnbc['date']=pd.to_datetime(cnbc['date']) #datetime 변환

cnbc[cnbc['contents']=='nan'] = np.nan
cnbc.dropna(subset=['contents'],how='any',axis=0,inplace=True) #결측치 제거

#cnbc.contents=cnbc.contents.apply(lambda x : text_preprocessing(x))   

all_week_cnbc=cnbc[cnbc.date.between('2022-08-18','2022-08-29')].sort_values('date') #원하는 날짜 조정 (주간 단위)
all_week_cnbc=all_week_cnbc.reset_index(drop=True)



In [ ]:
#형태소 분석 모델 다운로드
!python -m spacy download en_core_web_lg

In [ ]:
#형태소 분석 모델 정의 (spacy)
nlp = spacy.load("en_core_web_lg")#trf 모델로 변경가능
spacy_stopwords_list = list(nlp.Defaults.stop_words)
nlp.max_length = 40000000

nlp.get_pipe('attribute_ruler').add([[{"TEXT":"Russias"}]],{"LEMMA":"Russia"}) #굳이 안써도 됨.
nlp.get_pipe('attribute_ruler').add([[{"TEXT":"Ukraines"}]],{"LEMMA":"Ukraine"})
nlp.get_pipe('attribute_ruler').add([[{"TEXT":"Russians"}]],{"LEMMA":"Russian"})
nlp.get_pipe('attribute_ruler').add([[{"TEXT":"Ukrainians"}]],{"LEMMA":"Ukrainian"})

#추가 불용어리스트 정의하고자 하면 사용
stop_words_list=["year","u","cnbc","cnbcs","investor","s","t","p","m","c","k","cramer","cramers","jim","inc","q","fy","b","yahoo","bloomberg"]
for word in stop_words_list:
    spacy_stopwords_list.append(word)

In [ ]:
#vectorizer_model = CountVectorizer(stop_words=stopwords) 
#vectorizer_model = TfidfVectorizer(max_df=0.9)# countvectorizer과 선택해서 사용

umap_model = UMAP(n_neighbors=10, min_dist=0.1, n_components=4,random_state=42,metric='cosine') #umap_model 정의 
hdbscan_model=hdbscan.HDBSCAN(min_cluster_size=4, metric='euclidean',cluster_selection_method='eom', prediction_data=True)#hdbscan_model 정의

embedding_model = tensorflow_hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
# nr_topics ='auto' auto 제거시 더 많은 토픽을 볼 수 있음 (auto시 자동으로 유사도가 높은 토픽끼리 합침)
topic_model = BERTopic(verbose=True, embedding_model=embedding_model,top_n_words=15,min_topic_size=5,umap_model=umap_model,hdbscan_model=hdbscan_model)
#데이터 좀 많을 떄 min_topic_size=10 , min_cluster_Size=6 등으로 조절

In [ ]:
# False로 할시 속도 향상
topic_model.calculate_probabilities = True  

In [ ]:
topics, probabillities= topic_model.fit_transform(week_CNBC.contents)

In [ ]:
#(spacy) tag_ N으로 시작하고 정의한 불용어 사전에 포함되지 않고 entity에서 TIME, CARDINAL, DATE, PERSON은 제거 
#그리고 소문자로 변환해서 조건을 만족하는 값만 반환
#output을 뽑아낸 topics 변수에서 사후 텍스트 전처리 진행 
def tokenizer(text):
    words=[]
    doc=nlp(text)
    for token in doc:
        if token.tag_[0] in ['N','V'] and token.lemma_.lower() not in ENGLISH_STOP_WORDS and token.ent_type_ not in ['TIME','CARDINAL','DATE','PERSON']:
                if len(token.lemma_.lower())>1:
                    words.append(token.lemma_.lower())
                
    return words


In [ ]:
vectorizer_model = TfidfVectorizer(tokenizer=tokenizer,stop_words="english", ngram_range=(1, 2),min_df=2)
topic_model.update_topics(week_CNBC.contents, topics, vectorizer_model=vectorizer_model)#위에서 정의한 사용자 정의 tokenzier를 활용해 불용어 제거

In [ ]:
topic_model.get_topic_info() #전체 정보

In [ ]:
topic_model.get_topic(9)# 토픽 하나씩 뽑아서 키워드 체크

In [ ]:
similar_topics, similarity = topic_model.find_topics("moderna", top_n=5); similar_topics # 특정단어와 가장 연관성 높은 토픽 찾기 

In [ ]:
topic_model.get_representative_docs(14)[2] # 토픽에 속한 문서 내용 최대 3개까지 볼 수 잇음

In [ ]:
#bertopic 새로운 기능 hierarchical_topics를 활용해 비슷한 topic끼리는 묶어주자!
from scipy.cluster import hierarchy as sch

In [ ]:
linkage_function = lambda x: sch.linkage(x, 'single', optimal_ordering=True)

In [ ]:
hierarchical_topics = topic_model.hierarchical_topics(week_CNBC.contents, topics,linkage_function=linkage_function)

In [ ]:
tree = topic_model.get_topic_tree(hierarchical_topics)

In [ ]:
print(tree)

In [ ]:
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

In [ ]:
#토픽 merge
topics_to_merge = [32, 33]
topic_model.merge_topics(week_CNBC.contents, topics, topics_to_merge)

In [ ]:
#32,33번 토픽인 트위터가 하나로 합쳐진 것을 볼 수 있다.
topic_model.get_topic_info() #전체 정보

In [ ]:
topics_to_merge = [[1, 2]
                   [3, 4]]
topic_model.merge_topics(docs, topics, topics_to_merge)

In [ ]:
#calculate_probabilities =True 로 계산시 활용가능 -1로 분류된 문서들을 가장 연관있는 토픽으로 재분류
import numpy as np
probability_threshold = 0.01
new_topics = [np.argmax(prob) if max(prob) >= probability_threshold else -1 for prob in probabillities]

In [ ]:
week_CNBC['topics']=new_topics
topic_df=topic_model.get_topic_info() #전체 정보

In [ ]:
small_cnbc=week_CNBC.loc[:,['link','contents','topics']]

In [ ]:
topic_first=pd.merge(all_week_cnbc,small_cnbc,how='left',left_on=['link','contents'],right_on=['link','contents'])

In [ ]:
last=pd.merge(topic_first,topic_df,how='left',left_on='topics',right_on='Topic')

In [ ]:
save=pd.merge(week_CNBC,topic_df,how='left',left_on='topics',right_on='Topic')

In [ ]:
last.to_csv('0825_topics.csv',encoding='utf-8-sig')